In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

import nltk
import string
import re

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hafez\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Hafez\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
def ingest_data():
    from pathlib import Path

    file_list_pos = [path for path in Path('imdb').rglob('*/pos/*.txt')]
    file_list_neg = [path for path in Path('imdb').rglob('*/neg/*.txt')]

    corpus_positive = [open(f, encoding="utf8").read() for f in file_list_pos]
    corpus_negative = [open(f, encoding="utf8").read() for f in file_list_neg]

    df = pd.DataFrame(corpus_positive, columns=['text'])
    df['sentiment'] = 'pos'

    df2 = pd.DataFrame(corpus_negative, columns=['text'])
    df2['sentiment'] = 'neg'

    df = df.append(df2)
    df.to_pickle('reviews.pkl')

df = pd.read_pickle('reviews.pkl')


In [18]:
sw = set(nltk.corpus.stopwords.words('english'))

def clean(text, stopwords):
    # remove tags like <tab>    
    text = re.sub(r'<[^<>]*>', ' ', text)  
    # split text on whitespace
    text_list = text.split()
    text_words = []
    
    punctuation = set(string.punctuation)
    
    # keep #tags and @mentions 
    ## punctuation.remove("#")
    ## punctuation.remove("@")
    
    for word in text_list:
        # remove punctuation marks at the beginning  
        # of each word
        while len(word) > 0 and word[0] in punctuation:
            word = word[1:]
        
        # remove punctuation marks at the end of each word
        while len(word) > 0 and word[-1] in punctuation:
            word = word[:-1]
        
        # a rule to eliminate most urls 
        if len(word) > 0 and "/" not in word:
            # eliminate stopwords 
            if word.lower() not in stopwords:
                # append the word to the text_words list
                text_words.append(word.lower())
        cleaner_text = " ".join(text_words)
    return cleaner_text

df['length'] = df.text.str.len()
df = df[df.length >= 100]
df.drop(['length'], axis=1, inplace=True)

print(df.iloc[0])
df['text'] = df['text'].apply(clean, stopwords=sw)
print(df.iloc[0])

C:\Users\Hafez\anaconda3\envs\comp4448\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


text         I went and saw this movie last night after bei...
sentiment                                                  pos
Name: 0, dtype: object
text         went saw movie last night coaxed friends mine ...
sentiment                                                  pos
Name: 0, dtype: object


C:\Users\Hafez\AppData\Local\Temp/ipykernel_4156/2276190142.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(clean, stopwords=sw)


In [19]:
df.to_pickle('reviews.pkl')
